# HW2


Before submitting your **HTpy-HW2.ipynb** file, perform the following
* `Kernel>Restart & Run All`
* Make sure that there are no errors

The following includes problems that need coding and others that only need to perform simple operations (python as a calculator).

In [1]:
from Libraries import thermodynamics as thermo
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as csts

## 1 Convection heat transfer coefficient
A plate of metal of mass $m=4.25 kg$, specific heat $C_p= 2770 J/kg.K$ and surface area $0.4m\times0.4m$ is cooled in ambient air at temperature $T_\infty=25^\circ C$. The plate is thin enough and its thermal conductivity high enough that the plate is assumed to be isothermal at any time. At a given time, the plate temperature is $T_s = 245^\circ C$ and the rate of temperature cooling is
$$
\frac{dT}{dt}=-0.028 K/s
$$

Find the convection heat transfer coefficient from conservation of energy applied to the plate.

### Solution
Assumptions:
* Plate is isothermal
* Only heat transfer is convection heat transfer
* Conservation of energy is applied to the plate
* The plate is thin therefore the area exposed to convection is $A=2\times0.4^2m^2$

Conservation of energy applied to the metal plates writes:
$$
\frac{d E_{st}}{dt}=\dot{E}_{in} - \dot{E}_{out} + \dot{E}_g
$$

reduces to

$$
\frac{d E_{st}}{dt}=- \dot{E}_{out}
$$

where $\dot{E}_{out}$ is the thermal energy lost by convection to the ambient air:
$$
\dot{E}_{out}=q_{conv}=hA(T_s-T_\infty).
$$

Since $T_s=T$, the conservation of energy becomes:
$$
\frac{d}{dt}\int_V\rho C_pT dV=-hA(T-T_\infty)
$$

$$
mC_p\frac{dT}{dt}=-hA(T-T_\infty)
$$

Therefore,

$$
\frac{dT}{dt}=-\frac{hA}{mC_p}(T-T_\infty)
$$

or

$$
h=-\frac{mC_p}{A(T-T_\infty)}\frac{dT}{dt}
$$



In [2]:
Cp = 2770. #J/kg.K
m = 4.25 #kg
dTdt = -0.028 #K/s
T_infty = 25. #C
T_s = 245. #C
A = 2*0.4**2

h = -m*Cp/(A*(T_s - T_infty))*dTdt
print ("The convection heat transfer for the given conditions is %.1f W/m^2.K." %h)

The convection heat transfer for the given conditions is 4.7 W/m^2.K.


## 2 Radiation in a vacuum 

The same plate as above is now in a vacuum (no flow). Using all conditions defined in 1, determine the emissivity of the plate and the heat rate at which radiation is emitted from the plate. Assume that $T_{sur}=T_\infty$. 

### Solution
Assumptions:
* Same as above, except no convection

Using the same approache as above, with $\dot{E}_{out}=q_{rad}=\sigma\varepsilon A(T_s^4-T_\infty^4)$, the emissivity is
$$
m C_p\frac{dT}{dt}=\sigma\varepsilon A(T_s^4-T_\infty^4)
$$
$$
\varepsilon=-\frac{mC_p}{\sigma A(T_s^4-T_\infty^4)}\frac{dT}{dt}
$$

Emissive power of the plate is 

$$
q_{rad}=\varepsilon \sigma A T_s^4
$$

In [3]:
print('The Stefan-Boltzmann constant is available from scipy %.2e W/m^2.K^4' %csts.sigma)

The Stefan-Boltzmann constant is available from scipy 5.67e-08 W/m^2.K^4


In [4]:
Cp = 2770. #J/kg.K
m = 4.25 #kg
dTdt = -0.028 #K/s
T_infty = 25. #C
T_s = 245. #C
A = 2*0.4**2
eps = -m*Cp/(csts.sigma*A*(thermo.C2K(T_s)**4 - thermo.C2K(T_infty)**4))*dTdt
print('The emissivity of the plate is %.2f.' %eps)
q_rad = eps*csts.sigma*A*thermo.C2K(T_s)**4
print('The heat rate of radiation emitted by the plate is %.0f W.' %q_rad)

The emissivity of the plate is 0.28.
The heat rate of radiation emitted by the plate is 370 W.


## 3 Surface temperature of steam line

A steam line of diameter $D=0.1m$ and length $L=25m$ loses $18.405 kW$ to the ambient air. The air is at $T_\infty=T_{sur}=25^\circ C$, the convection coefficient of the air flow is $h=10W/m^2.K$ and the emissivity of the pipe is $\varepsilon=0.8$.

Find the surface temperature of the steam line, the annual energy loss and the annual cost assuming that the steam is generated by a furnace with an efficiency $\eta_f = 0.9$ running at a cost of natural gas of  $0.02\$/MJ$.

For the surface temperature, use `scipy.optimize.fsolve`

### Solution
Assumptions:
* Heat transfer is steady and 1D
* Conservation of energy is applied to the surface of the pipe. There is no energy generation in that control volume. 

The control volume is the external surface of the pipe of surface area $A=\pi D\times L=2.5m^2$. The steam loses heat to the outside at a rate $q=18.405kW$. This energy is transferred to the outside through radiation and convection. The conservation of energy applied to the surface of the pipe therefore reduces to:

$$
\dot{E}_{in}-\dot{E}_{out}=0
$$

with $\dot{E}_{in}=q$ and $\dot{E}_{out}=q_{conv}+q_{rad}$. For a surface temperature $T_s$, this equation becomes

$$
q-hA(T_s-T_\infty)-\sigma\varepsilon A(T_s^4-T_\infty^4)=0
$$

that we solve for $T_s$ using the function root finder solver fsolve from scipy.optimize.

The annual energy loss $E_{loss}$ is q multiplied by a year. The energy consumed by the boiler to create the total energy loss is
$$
E_{furnace}=\frac{E_{loss}}{\eta_f}
$$

Warning: the cost is in dollars per MEGAJoule.

with 
$$
\dot{E}_{in}=q_{loss} = 18.405kW
$$

$$
\dot{E}_{out}=q_{conv}+q_{rad}
$$

In [5]:
from scipy.optimize import fsolve
D = 0.1
L = 25.
q = 18405.
h = 10.
T_infty = 25.
eps = 0.8
eta_f = 0.9
Cost = 0.02
A = np.pi*D*L

def heatfunc(Ts):
    global q, h, A, T_infty, eps
    return q - h*A*(Ts-T_infty) - eps*csts.sigma*A*(thermo.C2K(Ts)**4 - thermo.C2K(T_infty)**4)
root = fsolve(func=heatfunc,x0=[100.])
T_s = root[0]
print('The steam line surface temperature is %.0f C.' %T_s)
print('verification:',A*h*(T_s-T_infty) + eps*csts.sigma*A*(thermo.C2K(T_s)**4-thermo.C2K(T_infty)**4))
cost = q*(365*24*3600)/0.9*Cost*1e-6
print('The cost of operation for a year is $%.f.' %cost)

The steam line surface temperature is 150 C.
verification: 18404.999999999996
The cost of operation for a year is $12898.


## 4 Insulation

Assuming that the surface temperature is $150^\circ C$, find an insulation that works (i.e. an insulation that will not melt, for which you can retrieve the price and thermal conductivity). Derive the necessary equations that will give optimum savings.

Hint: Derive the heat loss for a variable thickness of insulation, then apply this formula for a range of thicknesses. Compare savings of gas vs cost of insulation.

### Solution
Assumptions:

 

* Losses due to radiation are negligible.
* Heat transfer is steady and 1D.

 

The control volume is the hollow cylinder bounded by the external surface of the pipe and the internal surface of the insulation. The surface area is given by $A=\pi D\times L=2.5m^2$. Heat from the pipe is lost to the ambient air through the insulation by convection and conduction. The conservation of energy is applied to the hollow cylinder and reduces to:

 

The conservation of energy is:
$$
\dot{E}_{in}-\dot{E}_{out}=0
$$

 

 

$$
q - h(T_s - T_{\infty}) - k(2\pi r L)\frac{T_s - T{\infty}}{dr} = 0
$$

 

From the conservation of energy, the rate of heat loss to the surroundings is equal to the rate of heat transfer from the pipe to the insulation. 

 

Accounting for the efficiency of the boiler, the total energy loss per year is given by:

 

$$
E_{loss} = \frac{E_{loss}}{\eta_f} = \frac{h(T_s - T_{\infty}) - k(2\pi r L) \frac{T_s - T_{\infty}}{dr}}{\eta_f} * 1 year *\frac{365 day}{1 year} * \frac{24 hours}{1 day} * \frac{3600 sec}{1 hour}
$$

 

The cost of operating the boiler is calculated by converting the Joules to Mega-Joules and multiplying by the cost per Mega-Joule and given by the equation:

 

$$
\frac{cost}{year} = \frac{$0.02}{MJ}*E_{loss}
$$

 

The cost of material is given by:

 

$$
\frac{price}{foot}*L_{pipe}
$$

In [6]:
r1 = 0.05 #m 
L = 25 #mm
h = 10 #W/m**2
eta_f = 0.9
t_s = 150 #degC
T_infty = 25 #degC
k = .29
x = np.array([511.94,689.26,866.84])

for i in range(2,5):
    
    dr = i * .0254 / 2
    cost_fur = -(.02/10**6)*( h*(T_s - T_infty) - k*(2*np.pi*(dr/2 + r1)*L) * (T_s - T_infty)/dr * (365 * 24 * 3600) / eta_f)
    print("For %.3f inch thick insulation the cost of operation is $%.2f and the cost of material is $%.2f.\n" % (dr, cost_fur, x[i-2]))
    
#https://www.grainger.com/search/plumbing/insulation/pipe-insulation?tv_optin=true&searchQuery=closed+cell+glass+pipe+insulation&searchBar=true

For 0.025 inch thick insulation the cost of operation is $9842.06 and the cost of material is $511.94.

For 0.038 inch thick insulation the cost of operation is $7225.88 and the cost of material is $689.26.

For 0.051 inch thick insulation the cost of operation is $5917.79 and the cost of material is $866.84.



## 5 Window

Set up calculations to discuss heat losses of a single pane window, a single pane window with storm window and a double paned window with Argon vacuum.

### Solution

Assumptions:

* Heat transfer is steady and 1D.
* Air on both sides of window is static.
* Radiation is considered negligable.
* Window is considered a plane wall.
* Inside and outside windows are both made of glass and have the same thickness.

Resistance being the ratio of a driving potential to the corresponding transfer rate; thermal resistance of conduction is the resistance of heat trasfer through the body.

$$
R_{t,cond} = \frac{T_{s,1}-T_{s,2}}{q_x} = \frac{L}{kA}
$$

The single pane window with a stor window as well as the double paned window with Argon vacuum are both to be considered composite walls.

1D heat transfer rate for this system may be expressed as:

$$
q_x = \frac{T_{\infty,1}-T_{\infty,4}}{\Sigma R_t}
$$

Where the numerator is the overall temperature difference, and the summation includes all thermal resistances.
Expanded:

$$
q_x = \frac{T_{\infty,1}-T_{\infty,4}}{[(\frac{L_A}{k_AA})+(\frac{L_B}{k_BA})+(\frac{L_C}{k_CA})]}
$$

With composite systems, it is often convienent to work with an overall heat transfer coefficient U, which is defined by an expression analogus to Newton's law of cooling.

$$
q_x = UA\Delta T
$$

Where $\Delta T$ is the overall temperature difference. The overall heat transfer coefficient is related to the total thermal resistance, and the equations shown above, we see that
$$
UA = \frac{1}{R_{tot}}
$$

Therefore

$$
U = \frac{1}{R_{tot}A}= \frac{1}{[(\frac{L_A}{k_A})+(\frac{L_B}{k_B})+(\frac{L_C}{k_C})]}
$$

In [ ]:
k_ar = .016 #W/m*K
k_air = 1.4 #W/m*K
k_glass = .96 #W/m*K

print('What is the inside temperature (deg C)?')
T_1 = float(input())
print('What is the outside temperature (deg C)?')
T_4 = float(input())
print('What is the height of the window (m)?')
l = float(input())
print('What is the width of the window (m)?')
w = float(input())
print('How thick is the window? (m)')
L_ac = float(input())

print('Is there a storm window? Use 1 for yes and 0 for no:')
tf = float(input())

if tf == 1:
    print('How wide is the gap between the windows (m)?')
    L_b = float(input())
    print('Is the space filled with argon? Use 1 for yes and 0 for no:')
    tf = float(input())
    if tf == 1:
        U = 1 / (2*L_ac/k_glass + L_b/k_ar)
    else:
        U = 1 / (2*L_ac/k_glass + L_b/k_air)
        
else:
    U = 1 * k_glass / L_ac
    
q_x = U * l * w * (T_1 - T_4)

print('\nThe heat transfer rate through a window with these dimensions and properties will be %.3fW.' % q_x)
 

What is the inside temperature (deg C)?
